In [ ]:
from biocrnpyler import *
from genelet import *
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

In [13]:
# Creating CRN for NOR gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

S1 = Genelet(S1_off, transcript = "I3", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I3", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A3", inhibitor = "I3" )

M_NOR = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

repr(M_NOR)
CRN_NOR = M_NOR.compile_crn()
CRN_NOR.write_sbml_file('NOR_CRN.xml')
#print(CRN_NOR)

True

In [19]:
##### Creating atc reservoir model####


rI4_ss, rI4_model = createBasicSubsystem('external_reservoir')

per_second = rI4_model.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = rI4_model.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)

rI4_model.createNewSpecies('rna_I4',sComp = 'external_reservoir', ListOfAmounts = 0, 
                             sConstant = False, sSubstance = 'substance')

# Write to SBML file 
exsub = rI4_ss.writeSBML('rI4_external_reservoir.xml')
print(rI4_ss)

In [20]:
mb_ss = createNewSubsystem()
model = mb_ss.createNewModel('membrane','second','mole','substance')

mb_md = SimpleModel(model)

per_second = mb_md.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = mb_md.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
mb_md.createNewCompartment('internal','internal',1e-6,'litre',True)
mb_md.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

mb_md.createNewSpecies( 'rna_I4','external',0,False,'substance')
mb_md.createNewSpecies( 'rna_I4','internal',0,False,'substance')

mb_md.createNewParameter('kd', 2e-3, False, 'per_second')
mb_md.createNewParameter('kb', 2e-3, False, 'per_second')

# model = IPTG_mb_ss.getSBMLDocument().getModel()
r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('rna_I4_1',False,1)
r1.createNewProduct('rna_I4', False, 1)
r1.createRate('kb * rna_I4_1 - kd * rna_I4')

# Write to SBML file 
_ = mb_ss.writeSBML('membrane.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


In [21]:
##### Creating a advanced membrane model for IPTG transport####
complex_mb_ss = createNewSubsystem()
model = complex_mb_ss.createNewModel('complex_membrane','second','mole','substance')

complex_mb = SimpleModel(model)

per_second = complex_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = complex_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
complex_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
complex_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

complex_mb.createNewSpecies( 'rna_I4','external',0,False,'substance')
complex_mb.createNewSpecies( 'rna_I4','internal',0,False,'substance')
complex_mb.createNewSpecies( 'transport_protein','internal',1,False,'substance')
complex_mb.createNewSpecies( 'rna_I4_transport_protein','internal',0,False,'substance')

complex_mb.createNewParameter( 'kb',0.01,False,'per_second')
complex_mb.createNewParameter( 'kd',0.2,False,'per_second')

model = complex_mb_ss.getSBMLDocument().getModel()

r1 = SimpleReaction(model.createReaction())
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('rna_I4',False,1)
r1.createNewReactant('transport_protein', False, 1)
r1.createNewProduct('rna_I4_transport_protein', False, 1)
r1.createRate('kb * rna_I4 * transport_protein - kd * rna_I4_transport_protein')


r2 = SimpleReaction(model.createReaction())
r2.setId('r2')
r2.setReversible(True)
r2.createNewReactant('rna_I4_transport_protein',False,1)
r2.createNewProduct('rna_I4_1',False,1)
r2.createNewProduct('transport_protein',False,1)
r2.createRate('kd * rna_I4_transport_protein - kb * rna_I4_1 * transport_protein')


# Write to SBML file 
_ = complex_mb_ss.writeSBML('membrane_detailed.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully
